# 轨迹生成算法

In [8]:
import shapefile
from shapely.geometry import MultiLineString
import numpy as np
import pandas as pd
from anytree import AnyNode
from anytree.search import findall_by_attr

import matplotlib.pyplot as plt
import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs

## 输入端口

In [9]:
# 点击按键
def con(x):
    global prop_lst, geom_lst
    ## 读取文件
    fi = shapefile.Reader(f'workingdata5/{out_way.value}.shp')
    records = pd.DataFrame(fi.records(),columns=[i[0] for i in fi.fields[1:]])  # 用于加权计算
    geoms = fi.shapes()  # 地理信息
    fi.close()
    adjacent = np.load(f'workingdata5/{out_way.value}.npy',allow_pickle=True).item()

    ## 要素阈值与距离阈值
    # 连结要素
    link_dict = {'热门程度':'hot','服务设施完善程度':'serve','景观类型-人文':'scene_h','景观类型-自然':'scene_n','挑战性':'dem'}
    link_field = link_dict[link_option.value]
    link_tol_cls = 12 - link_cls.value  # 连接要素-距离阈值
    link_tol_sep = 11 - link_sep.value
    link_tol_sep_on = link_sep_on.value

    link_field2_on = link_option2_on.value
    link_field2 = link_dict[link_option2.value]
    link_tol2_cls = 12 - link_cls2.value
    link_tol2_sep = 11 - link_sep2.value
    link_tol2_sep_on = link_sep2_on.value
    # 筛选要素
    datetime = {'今天':'0','明天':'1','后天':'2'}  # 筛选要素-要素阈值
    like_short, like_long = like_length.value

    ## 要素二值化
    # 连结要素
    records['link'] = records[link_field]
    records['link2'] = records[link_field2]
    # 筛选要素
    records['choose_weather'] = records['weather'+datetime[out_day.value]]
    records['choose_length'] = records['SHAPE_Leng']
    # 评价要素
    records['star'] = records['hot']*weight_hot.value + records['serve']*weight_serve.value + records['dem']*weight_dem.value + records['weather'+datetime[out_day.value]]*weight_weather.value

    ## 拼接
    rank = 0
    geom_lst = []
    prop_lst = []
    nodes_set = set()  # 是否已遍历
    if link_field2_on:
        # 索引结构
        for root_idx in np.where((records['link'] == 1))[0]:
            if root_idx in nodes_set:  # 若已遍历则跳过
                continue
            root = AnyNode(name=str(root_idx) + "_0_0")
            cdts = [root]

            while len(cdts) > 0:
                current_node_idx = int(cdts[0].name.split("_")[0])  # 当前小节索引
                adjacent_lines_idx = adjacent[current_node_idx]  # 临近小节索引-列表
                nodes_set.add(current_node_idx)  # 路线小节索引-组
                # 判断小节可行性
                for adjacent_line_idx in adjacent_lines_idx:
                    # 判断要素阈值可行性
                    flag = int(records.loc[adjacent_line_idx, 'link'])
                    flag2 = int(records.loc[adjacent_line_idx, 'link2'])
                    node_name = "%d_%d_%d" % (adjacent_line_idx, flag, flag2)
                    # 构建路线列表
                    current_path_cls = sum([int(node.name.split("_")[1]) for node in list(cdts[0].iter_path_reverse())[:link_tol_cls]]) + int(flag)
                    current_path_sep = sum([int(node.name.split("_")[1]) for node in list(cdts[0].iter_path_reverse())[:link_tol_sep]]) + int(flag)
                    current_path2_cls = sum([int(node.name.split("_")[2]) for node in list(cdts[0].iter_path_reverse())[:link_tol2_cls]]) + int(flag2)
                    current_path2_sep = sum([int(node.name.split("_")[2]) for node in list(cdts[0].iter_path_reverse())[:link_tol2_sep]]) + int(flag2)                    # 判断距离阈值可行性：测度当前不符合条件连续小节数
                    if len(list(findall_by_attr(root, node_name))) < 1 \
                            and current_path_cls > 0 \
                            and (not link_tol_sep_on or current_path_sep < link_tol_sep)\
                            and current_path2_cls > 0 \
                            and (not link_tol2_sep_on or current_path2_sep < link_tol2_sep)\
                            and adjacent_line_idx not in nodes_set:
                        adjacent_line = AnyNode(name=node_name, parent=cdts[0])
                        nodes_set.add(adjacent_line_idx)
                        cdts.append(adjacent_line)
                # 筛选要素筛选
                if len(cdts) > like_long*20 or records.loc[current_node_idx,'choose_weather'] == 0:
                    break
                del cdts[0]

            nodes = set()
            dfs_paths = [leaf.iter_path_reverse() for leaf in root.leaves]
            for path in dfs_paths:
                flag = False
                for node in path:
                    if node.name.split("_")[1] == "1" or (link_field2_on and node.name.split("_")[2] == "1"):
                        flag = True
                    if flag:
                        nodes.add(node)

            # 筛选要素筛选
            total_length = sum([records.loc[int(node.name.split("_")[0]),'choose_length'] for node in list(nodes)])
            if total_length < like_short*1000:
                continue

            # 评分信息计算
            total_star = sum([records.loc[int(node.name.split("_")[0]),'star'] for node in list(nodes)])
            star = round(total_star / total_length,3)
            # 其他信息计算
            total_element_lst = sum([records.loc[int(node.name.split("_")[0]),['hot','serve','dem','choose_weather']] for node in list(nodes)])
            element_lst = [round(total_element / total_length,3) for total_element in total_element_lst]

            # 路线/信息写入列表
            geom_lst.append(MultiLineString([geoms[int(node.name.split("_")[0])].points for node in list(nodes)]))  # 路线
            prop_lst.append([rank, star, total_length]+element_lst)  # 信息
            rank += 1
    else:
        # 索引结构
        for root_idx in np.where((records['link'] == 1))[0]:
            if root_idx in nodes_set:  # 若已遍历则跳过
                continue
            root = AnyNode(name=str(root_idx) + "_0")
            cdts = [root]

            while len(cdts) > 0:
                current_node_idx = int(cdts[0].name.split("_")[0])  # 当前小节索引
                adjacent_lines_idx = adjacent[current_node_idx]  # 临近小节索引-列表
                nodes_set.add(current_node_idx)  # 路线小节索引-组
                # 判断小节可行性
                for adjacent_line_idx in adjacent_lines_idx:
                    # 判断要素阈值可行性
                    flag = int(records.loc[adjacent_line_idx, 'link'])
                    node_name = "%d_%d" % (adjacent_line_idx, flag)
                    # 构建路线列表
                    current_path_cls = sum([int(node.name.split("_")[1]) for node in list(cdts[0].iter_path_reverse())[:link_tol_cls]]) + int(flag)
                    current_path_sep = sum([int(node.name.split("_")[1]) for node in list(cdts[0].iter_path_reverse())[:link_tol_sep]]) + int(flag)
                    # 判断距离阈值可行性：测度当前不符合条件连续小节数
                    if len(list(findall_by_attr(root, node_name))) < 1 \
                            and current_path_cls > 0 \
                            and (not link_tol_sep_on or current_path_sep < link_tol_sep)\
                            and adjacent_line_idx not in nodes_set:
                        adjacent_line = AnyNode(name=node_name, parent=cdts[0])
                        nodes_set.add(adjacent_line_idx)
                        cdts.append(adjacent_line)
                # 筛选要素筛选
                if len(cdts) > like_long*20 or records.loc[current_node_idx,'choose_weather'] == 0:
                    break
                del cdts[0]

            nodes = set()
            dfs_paths = [leaf.iter_path_reverse() for leaf in root.leaves]
            for path in dfs_paths:
                flag = False
                for node in path:
                    if node.name.split("_")[1] == "1":
                        flag = True
                    if flag:
                        nodes.add(node)

            # 筛选要素筛选
            total_length = sum([records.loc[int(node.name.split("_")[0]),'choose_length'] for node in list(nodes)])
            if total_length < like_short*1000:
                continue

            # 评分信息计算
            total_star = sum([records.loc[int(node.name.split("_")[0]),'star'] for node in list(nodes)])
            star = round(total_star / total_length,3)
            # 其他信息计算
            total_element_lst = sum([records.loc[int(node.name.split("_")[0]),['hot','serve','dem','choose_weather']] for node in list(nodes)])
            element_lst = [round(total_element / total_length,3) for total_element in total_element_lst]

            # 路线/信息写入列表
            geom_lst.append(MultiLineString([geoms[int(node.name.split("_")[0])].points for node in list(nodes)]))  # 路线
            prop_lst.append([rank, star, total_length]+element_lst)  # 信息
            rank += 1

    prop_lst = pd.DataFrame(prop_lst,columns=['rank','综合评分','总长度','热门程度','服务设施完善程度','挑战性','气候状况'])  # 简单排序

In [10]:
import ipywidgets as widgets
widgets.HTML(value='要素权重偏好')

# 出行类型
a = widgets.HTML(value='<h2>出行方式</h2><hr>')
out_way = widgets.Dropdown(options=['徒步','登山','骑行'],description='出行类型')
out_day = widgets.Dropdown(options=['今天','明天','后天'],description='出行日期')

# 连结要求
b = widgets.HTML(value='<h2>连结要求</h2><hr>')
link_option = widgets.Dropdown(options=['热门程度','服务设施完善程度','景观类型-人文','景观类型-自然','挑战性'],description='核心需求')
link_cls = widgets.IntSlider(min=1,max=5,description='紧凑度',value=3)
link_sep = widgets.IntSlider(min=1,max=5,description='松散度',value=3)
link_sep_on = widgets.Checkbox(value=False,description='是否使用指标（非挑战性慎用）')
link_option2 = widgets.Dropdown(options=['热门程度','服务设施完善程度','景观类型-人文','景观类型-自然','挑战性'],description='核心需求',value='服务设施完善程度')
link_option2_on = widgets.Checkbox(value=False,description='是否使用第二个要素')
link_cls2 = widgets.IntSlider(min=1,max=5,description='紧凑度',value=3)
link_sep2 = widgets.IntSlider(min=1,max=5,description='松散度',value=3)
link_sep2_on = widgets.Checkbox(value=False,description='是否使用指标（非挑战性慎用）')

# 筛选变量
c = widgets.HTML(value='<h2>筛选条件</h2><hr>')
like_length = widgets.SelectionRangeSlider(options=[2,3,4,5,6,7,8],description='线路长度km',value=(2,8))

# 要素间权重
d = widgets.HTML(value='<h2>评价权重</h2><hr>')
weight_hot = widgets.FloatSlider(min=0.2,max=1.8,step=0.4,description='热门程度',value=1,readout_format='.1f')
weight_serve = widgets.FloatSlider(min=0.2,max=1.8,step=0.4,description='服务设施完善程度',value=1,readout_format='.1f')
weight_dem = widgets.FloatSlider(min=0.2,max=1.8,step=0.4,description='挑战性',value=1,readout_format='.1f')
weight_weather = widgets.FloatSlider(min=0.2,max=1.8,step=0.4,description='天气状况',value=1,readout_format='.1f')

In [11]:
button = widgets.Button(description='开始构建')
button.on_click(con)

display(
    a, out_way,out_day,
    b, link_option,link_cls,widgets.HBox((link_sep,link_sep_on)),
    widgets.HBox((link_option2,link_option2_on)),link_cls2,widgets.HBox((link_sep2,link_sep2_on)),
    c, like_length,
    d, weight_hot,weight_serve,weight_dem,weight_weather,
    button
)

HTML(value='<h2>出行方式</h2><hr>')

Dropdown(description='出行类型', options=('徒步', '登山', '骑行'), value='徒步')

Dropdown(description='出行日期', options=('今天', '明天', '后天'), value='今天')

HTML(value='<h2>连结要求</h2><hr>')

Dropdown(description='核心需求', options=('热门程度', '服务设施完善程度', '景观类型-人文', '景观类型-自然', '挑战性'), value='热门程度')

...

FloatSlider(value=1.0, description='热门程度', max=1.8, min=0.2, readout_format='.1f', step=0.4)

FloatSlider(value=1.0, description='服务设施完善程度', max=1.8, min=0.2, readout_format='.1f', step=0.4)

FloatSlider(value=1.0, description='挑战性', max=1.8, min=0.2, readout_format='.1f', step=0.4)

FloatSlider(value=1.0, description='天气状况', max=1.8, min=0.2, readout_format='.1f', step=0.4)

Button(description='开始构建', style=ButtonStyle())

## 输出端口

In [12]:
from ipywidgets import interact_manual


@interact_manual
def sort(rank_element=['综合评分','总长度','热门程度','服务设施完善程度','景点评价','挑战性','气候状况'],
         ascending_element=['高值在前','低值在前']
        ):
    global prop_lst
    prop_lst = prop_lst.sort_values(by=rank_element,ascending=(ascending_element=='低值在前')).reset_index(drop=True)
    display('符合条件线路总数：'+str(len(prop_lst)),prop_lst.head(15))

interactive(children=(Dropdown(description='rank_element', options=('综合评分', '总长度', '热门程度', '服务设施完善程度', '景点评价',…

In [13]:
import cartopy.io.img_tiles as cimgt
request = cimgt.GoogleTiles()

In [14]:
from ipywidgets import interact_manual


@interact_manual
def geoplot(prop=range(0,15)):
    geom = geom_lst[prop_lst.loc[prop,'rank']]
    bound = geom.bounds
    bound_out = 2000

    # 创建小地图画布
    fig = plt.figure(figsize=(10, 10))
    ax1 = fig.add_subplot(1, 2, 1, projection=ccrs.UTM(zone=50))
    ax1.set_extent([bound[0]-bound_out,bound[2]+bound_out,bound[1]-bound_out,bound[3]+bound_out], crs=ccrs.UTM(zone=50))  # 设置显示范围
    # 添加网络地图到地图上，设置透明度为0.5
    ax1.add_image(request, 12)
    # 绘制北京边界
    reader = shpreader.Reader('../workingdata5/Beijing_Border1.shp')
    records = reader.records()
    for record in records:
        ax1.add_geometries([record.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='black',linestyle="--")  # 设置图例
    # 绘制内容
    # ax1.gridlines(draw_labels=True,linewidth=0,color='k')
    ax1.add_geometries(geom,ccrs.UTM(zone=50),lw=5, edgecolor='red')

    # 创建大地图画布
    ax2 = fig.add_subplot(1, 2, 2, projection=ccrs.UTM(zone=50))
    ax2.set_extent([115,117.75,39.25,41.25], crs=ccrs.PlateCarree())  # 设置显示范围
    # 添加地理特征
    ax2.add_feature(cartopy.feature.LAND)
    ax2.add_feature(cartopy.feature.LAKES, alpha=0.5)
    ax2.add_feature(cartopy.feature.RIVERS)
    # 绘制北京边界
    records = reader.records()
    for record in records:
        ax2.add_geometries([record.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='black',linestyle="--")  # 设置图例
    # 绘制大地图内容
    # ax2.gridlines(draw_labels=True,linewidth=0,color='k')
    ax2.add_geometries(geom,ccrs.UTM(zone=50),lw=4, edgecolor='red')

    plt.show()

interactive(children=(Dropdown(description='prop', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14),…